In [11]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
slim=tf.contrib.slim
if __name__ == "__main__":
    sys.path.append("../../../")
from notebooks_src.losses.utils import abs_smooth as smooth_L1

In [12]:

from notebooks_src.configs import configs
from notebooks_src.postprocessing.utils import get_int_tensor_shape, sort_some_lists_of_tensors

from notebooks_src.box_processing.match import match_boxes
from notebooks_src.box_processing.encode import encode
from notebooks_src.box_processing.unpack import unpack_net_output
#from utils import ssd_losses

In [13]:
loss_weights = None

In [14]:
def compute_loss(y_true, y_preds):
    '''y_true: the boxes Nx15x5 tensor
       y_preds: a list of 7?  tensors of Nxfy x fx x k where k = 4*number of anchors + number_of_anchors*num_classes,
       N is number of examples'''

    bboxes, labels = y_true[:,:,:4], y_true[:,:,4]
    encoded_boxes_dict, encoded_labels_dict = encode(bboxes,labels)
    x_mask_dict, num_matches_dict, actual_gt_box_mask = match_boxes(bboxes)
    loc_dict, log_dict, pred_dict = unpack_net_output(y_preds)
    loss = tf.constant(0.)
    for fmap_size in encoded_boxes_dict.keys():
        encoded_boxes, encoded_labels, x_mask,
        num_matches, loc, log, pred = [dic[fmap_size] for dic in [encoded_boxes_dict,
                                                          encoded_labels_dict,x_mask_dict, num_matches_dict,
                                                           loc_dict, log_dict, pred_dict ]]
        fmap_loss = calc_loss_one_layer(encoded_boxes, encoded_labels, x_mask,
                                        num_matches,actual_gt_box_mask, loc, log, pred )
        loss = loss + fmap_loss
    return loss
        
        

In [15]:
def calc_loss_one_layer(encoded_boxes, encoded_labels, x_mask, num_matches,actual_gt_box_mask, loc, log, pred ):
    loc_loss = calc_loc_loss(x_mask, encoded_boxes, loc)
    cls_loss = calc_cls_loss(x_mask, encoded_labels,log, pred,actual_gt_box_mask )
    return (loc_loss + cls_loss) * (1/num_matches) * (1/batch_size)

def calc_loc_loss(x_mask, encoded_boxes, loc):
    loc = tf.expand_dims(loc,axis=-1)
    x_mask = tf.stack(4*[x_mask], axis=-2)
    x_mask = tf.cast(x_mask, dtype=tf.float32)
    loc_loss = smooth_L1(encoded_boxes - loc) * x_mask
    loc_loss = tf.reduce_sum(loc_loss)
    return loc_loss

def calc_cls_loss(x_mask, encoded_labels,log, pred,actual_gt_box_mask ):
    x_mask = tf.cast(x_mask, dtype=tf.float32)
    log = tf.expand_dims(log,axis=-2)
    log = tf.stack(15*[log], axis=-2)
    xent_loss = tf.nn.softmax_cross_entropy_with_logits(logits=log,labels=encoded_labels)
    pos_xent = tf.reduce_sum(x_mask * xent_loss)
    #make a mask for most confident negatives, conf_neg_mask
    neg_xent = tf.reduce_sum(conf_neg_mask * xent_loss)
    
    return pos_xent + neg_xent
    
    
    

In [16]:
if __name__ == "__main__":
    with tf.Session() as sess:
        from notebooks_src.load_data.get_generator import get_generator

        gen=get_generator("tr", batch_size=2)
        bboxes = tf.placeholder(tf.float32,shape=(2,15,5),name="bboxes")
        shapes = [(2, 6, 9, 48),
                 (2, 3, 5, 32),
                 (2, 96, 144, 32),
                 (2, 24, 36, 48),
                 (2, 12, 18, 48),
                 (2, 48, 72, 48),
                 (2, 1, 1, 32)]

        y_preds = [tf.ones((shape)) for shape in shapes]

        final_loss = compute_loss(bboxes, y_preds)
        for im, box in gen:
            print sess.run(final_loss, feed_dict={bboxes:box})

importing Jupyter notebook from ../../../notebooks_src/load_data/get_generator.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/generator/generator.ipynb


ImportError: No module named box_encode_decode.np_util